In [4]:
import torch
import io

from torchtext.vocab import build_vocab_from_iterator
from torchtext.data.utils import ngrams_iterator
from torchtext.data.utils import get_tokenizer
from torchtext.utils import unicode_csv_reader

In [5]:
build_vocab_from_iterator.__module__

'torchtext.vocab.vocab_factory'

In [19]:
def csv_iterator(data_path, ngrams, column_indices):
    tokenizer = get_tokenizer("spacy")
    with io.open(data_path, encoding="utf8") as f:
        reader = unicode_csv_reader(f, delimiter="\t")
        for row in reader:
            row_iter = [row[i] for i in column_indices]
            tokens = ' '.join(row_iter)
            yield ngrams_iterator(tokenizer(tokens), ngrams)

In [20]:
unk_token = '<unk>'
vocab = build_vocab_from_iterator(csv_iterator("../datasets/systematic_review/phase1.train.shuf.tsv", 1, [2]), specials=["<unk>"])
vocab.set_default_index(vocab[unk_token])

C:\Users\ryanc\AppData\Local\pypoetry\Cache\virtualenvs\ir-classification-_Pgcz6ju-py3.9\lib\site-packages\torchtext\data\utils.py:123: UserWarning: Spacy model "en" could not be loaded, trying "en_core_web_sm" instead
  warnings.warn(f'Spacy model "{language}" could not be loaded, trying "{OLD_MODEL_SHORTCUTS[language]}" instead')


TypeError: '<' not supported between instances of 'generator' and 'str'

In [96]:
tokenizer = get_tokenizer("spacy")
text_pipeline = lambda x: vocab(tokenizer(x))
label_pipeline = lambda x: int(x) + 1


C:\Users\ryanc\AppData\Local\pypoetry\Cache\virtualenvs\ir-classification-_Pgcz6ju-py3.9\lib\site-packages\torchtext\data\utils.py:123: UserWarning: Spacy model "en" could not be loaded, trying "en_core_web_sm" instead
  warnings.warn(f'Spacy model "{language}" could not be loaded, trying "{OLD_MODEL_SHORTCUTS[language]}" instead')


In [ ]:
from torch.utils.data import DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_label, _text) in batch:
         label_list.append(label_pipeline(_label))
         processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
         text_list.append(processed_text)
         offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)

train_iter = csv_iterator(split='train')
dataloader = DataLoader(train_iter, batch_size=8, shuffle=False, collate_fn=collate_batch)

In [21]:
from torchtext.datasets import AG_NEWS
train_iter = AG_NEWS(split='train')

29.5MB [00:02, 13.9MB/s]


In [108]:
import io
from torchtext.utils import (
    unicode_csv_reader,
)
from typing import Callable, List
from torch.utils import data

class TSVClassificationDataset(data.IterableDataset):
    def __init__(self, filepath: str, data_columns: List[int], tokenizer: Callable):
        self._number_of_items = _get_tsv_file_length(filepath)
        self._iterator = _create_data_from_tsv(filepath, data_column_indices=data_columns)
        self._tokenizer = tokenizer
        self._current_position = 0


    def __iter__(self):
        return self

    def __next__(self):
        item = next(self._iterator)
        self._current_position += 1
        label, text = item
        return label+1, self._tokenizer(text)

    def __len__(self):
        return 100

def _create_data_from_tsv(data_path, data_column_indices):
    with io.open(data_path, encoding="utf8") as f:
        reader = unicode_csv_reader(f, delimiter="\t")
        for row in reader:
            data = [row[i] for i in data_column_indices]
            yield int(row[0]), ' '.join(data)

In [109]:
data = TSVClassificationDataset("../datasets/systematic_review/phase1.train.shuf.tsv", [2], tokenizer)

In [135]:
len(data._iterator)

TypeError: object of type 'generator' has no len()